In [48]:
import expectexception

from typing import List, Union, Optional

# Simple Input Processing

This is a small collection of "proper implementations" of basic incoming data transformations for basic types. As data is often coming in over the web in a textual form, these primarily target string transformations.

## Boolean Values

There are many "natural" ways to express if a value is on or true, false or off, and so forth. Over HTTP, type information is lost and all values are transferred as strings, your checkboxes will need an acceptable value to use. Additionally, there's an interesting trick for `<input type=checkbox>` inputs:

```
<input type=hidden name=example value=false>
<input type=checkbox name=example value=true>
```

In this way you can ensure if the checkbox is not checked, a value of `false` will be sent, and if it is, an array of `['false', 'true']` will be sent. The first will resolve to `False`, the second to `True`.

In [36]:
def boolean(input) -> bool:
	"""Convert the given input to a boolean value.
	
	Intelligently handles boolean and non-string values, returning as-is or passing to the bool builtin respectively.
	
	This process is case-insensitive.
	"""
	
	try:  # Eliminate unnecessary distractions and choice.
		input = input.strip().lower()
	except AttributeError:  # "Duck typing" — we aren't explicitly checking for "stringiness".
		return bool(input)  # Fall back to standard typecasting; handles bools, ints, &c.
	
	if input in ('yes', 'y', 'on', 'true', 't', '1'):
		return True
	
	if input in ('no', 'n', 'off', 'false', 'f', '0'):
		return False
	
	raise ValueError("Unable to convert {0!r} to a boolean value.".format(input))

In [37]:
boolean("yes")

True

In [38]:
boolean("f")

False

In [39]:
boolean(True)

True

In [40]:
boolean(False)

False

In [41]:
boolean(1)  # And so forth...

True

In [42]:
boolean(['false', 'true'])

True

In [43]:
%%expect_exception ValueError

boolean("hi")

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
<ipython-input-43-dcb8e8559682> in <module>
----> 1 boolean("hi")

<ipython-input-36-f304bacb583a> in boolean(input)
     18                 return False
     19 
---> 20         raise ValueError("Unable to convert {0!r} to a boolean value.".format(input))

ValueError: Unable to convert 'hi' to a boolean value.


## Delimited Lists

When an actual `list` of values can not be normally produced, a limited form of encoding or serialization using a delimiter can be used. This is typically a comma, sometimes a space, but may be any symbol found suitable.

Ultimately, you'll need the individual values composing the serialized one back as a list of separate values.

In [51]:
def array(input:Union[None,List[str],str], *, separator:Optional[str]=',', strip:bool=True, empty:bool=False) -> List:
	"""Convert the given input to a list.
	
	Intelligently handles list and non-string values, returning as-is and passing to the list builtin respectively.
	
	The default optional keyword arguments allow for lists in the form:
	
		"foo,bar, baz   , diz" -> ['foo', 'bar', 'baz', 'diz']
	
	The separator may be None to split on any whitespace.
	"""
	
	if input is None:
		return []
	
	if isinstance(input, list):
		if not empty:
			return [i for i in input if i]
		
		return input
	
	if not isinstance(input, str):
		if not empty:
			return [i for i in list(input) if i]
		
		return list(input)
	
	if not strip:
		if not empty:
			return [i for i in input.split(separator) if i]
		
		return input.split(separator)
	
	if not empty:
		return [i for i in [i.strip() for i in input.split(separator)] if i]
	
	return [i.strip() for i in input.split(separator)]

In [52]:
array("foo,bar, baz   , diz")

['foo', 'bar', 'baz', 'diz']

In [55]:
array("   ")

[]

In [56]:
array([1, 2])

[1, 2]

In [58]:
array([1, "", 0, 5])  # Notable!

[1, 5]

In [59]:
array([1, "", 0, 5], empty=True)

[1, '', 0, 5]

# Numeric Processing

A lot of input data you receive will likely be numeric in nature. But may still be transferred as a string! Optimal representation would simply be to pass that string to `Decimal`, which, interestingly, internally represents the number as a string. This doesn't really require a dedicated function, but one can be used to _harmonize_ the failure states.

In [60]:
def integer(input) -> int:
    """Convert the given input to an integer value."""
    
    try:
        return int(input)
    except (TypeError, ValueError):
        raise ValueError(f"Unable to convert {input!r} to an integer value.")

In [61]:
integer(27)

27

In [62]:
integer("42")

42

Many numbers won't be nice, whole integers, but if the number can be represented by one, an integer should be used. IEEE floating point numbers aren't entirely accurate, but are a common representitive format for decimal numbers.

In [63]:
def number(input) -> Union[int, float]:
	"""Convert the given input to a floating point or integer value.
	
	In cases of ambiguity, integers will be prefered to floating point.
	"""
	
	try:
		return int(input)
	except (TypeError, ValueError):
		pass
	
	try:
		return float(input)
	except (TypeError, ValueError):
		raise ValueError(f"Unable to convert {input!r} to a number.")

In [65]:
number(27)

27

In [67]:
number("42")

42

In [68]:
number("3.141593")

3.141593

In [69]:
%%expect_exception ValueError

number("3.1.4.1")

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
<ipython-input-63-bd798fc8f8f7> in number(input)
     12         try:
---> 13                 return float(input)
     14         except (TypeError, ValueError):

ValueError: could not convert string to float: '3.1.4.1'

During handling of the above exception, another exception occurred:

ValueError                                Traceback (most recent call last)
<ipython-input-69-026c2a1483c7> in <module>
----> 1 number("3.1.4.1")

<ipython-input-63-bd798fc8f8f7> in number(input)
     13                 return float(input)
     14         except (TypeError, ValueError):
---> 15                 raise ValueError(f"Unable to convert {input!r} to a number.")

ValueError: Unable to convert '3.1.4.1' to a number.
